## CS310 Natural Language Processing
## Assignment 3. Recurrent Neural Networks for Language Modeling 

**Total points**: 

In this assignment, you will train a vanilla RNN-based language model on the Harry Potter text data. 

### 0. Import Necessary Libraries

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import torch
import torch.nn as nn
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader, random_split
import logging
from datetime import datetime
import torch
import torch.nn.functional as F
import math

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /Users/ruiyuhan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/ruiyuhan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
def read_text(file_path):
    with open(file_path,'r',encoding='utf-8')as f:
        text=f.read()
    return text

text=read_text('Harry_Potter_all_books_preprocessed.txt')
text=text.lower()
words=word_tokenize(text)
print(words[:10])

['the', 'boy', 'who', 'lived', 'mr', 'and', 'mrs', 'dursley', 'of', 'number']


In [ ]:
def build_vocab(words):
    vocab={'<PAD>':0,'<UNK>':1}
    word_count=Counter(words)
    for word, _ in word_count.items():
        vocab[word]=len(vocab)
    return vocab

vocab=build_vocab(words)
print('the size of vocab:',len(vocab))

the size of vocab: 26355


In [6]:
def text_to_tensor(words,vocab):
    indices=[vocab.get(word,vocab['<UNK>']) for word in words]
    return torch.tensor(indices,dtype=torch.long)

data_tensor=text_to_tensor(words,vocab)
print("the shape of data_tensor",data_tensor.size())
print(data_tensor[:10])

the shape of data_tensor torch.Size([1105952])
tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11])


In [7]:
class TextDataset(Dataset):
    def __init__(self,data_tensor,seq_length):
        self.data=data_tensor
        self.seq_length=seq_length
    def __len__(self):
        return len(self.data)-self.seq_length
    def __getitem__(self,idx):
        input_seq = self.data[idx:idx + self.seq_length]
        target_seq = self.data[idx + 1:idx + self.seq_length + 1]
        return input_seq,target_seq

### 1. Build the Model

In [8]:
class RNN_LM(nn.Module):
    def __init__(self,vocab_size,embed_size,hidden_size,num_layers,bidirectional=False):
        super(RNN_LM,self).__init__()
        self.embed=nn.Embedding(vocab_size,embed_size)
        self.rnn=nn.RNN(embed_size,hidden_size,num_layers,batch_first=True,bidirectional=bidirectional)
        self.bidirectional=bidirectional
        self.num_layers=num_layers
        self.hidden_size=hidden_size
        if bidirectional:
            self.fc=nn.Linear(hidden_size*2,vocab_size)
        else:
            self.fc=nn.Linear(hidden_size,vocab_size)
    def forward(self,x,hidden=None):
        # x: [batch_size, seq_length]
        embedded=self.embed(x)
        output,hidden=self.rnn(embedded,hidden)
        output=self.fc(output)
        return output,hidden
    def init_hidden(self, batch_size, device):
        # 初始化隐藏状态
        # 对于单向 RNN: [num_layers, batch_size, hidden_size]
        # 对于双向 RNN: [num_layers * 2, batch_size, hidden_size]
        num_directions = 2 if self.bidirectional else 1
        hidden = torch.zeros(self.num_layers * num_directions, batch_size, self.hidden_size, device=device)
        return hidden

vocab_size=len(vocab)
embed_size=128
hidden_size=256
num_layers=2
bidirectional=False

rnn_model=RNN_LM(vocab_size,embed_size,hidden_size,num_layers,bidirectional)
print(rnn_model)

RNN_LM(
  (embed): Embedding(26355, 128)
  (rnn): RNN(128, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=26355, bias=True)
)


In [9]:
seq_length=20
dataset=TextDataset(data_tensor,seq_length)
total_size=len(dataset)
train_size=int(total_size*0.9)
test_size=int(total_size*0.05)
val_size=total_size-train_size-test_size
train_dataset,val_dataset,test_dataset=random_split(dataset,[train_size,val_size,test_size])

train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

print(f"size of train dataset: {len(train_dataset)}")
print(f"size of test dataset: {len(test_dataset)}")
print(f"size of val dataset: {len(val_dataset)}")

size of train dataset: 995338
size of test dataset: 55296
size of val dataset: 55298


### 2. Train and Evaluate

In [ ]:
def setup_logging():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_filename = f"training_log_{timestamp}.log"
    
    logger = logging.getLogger()
    for handler in logger.handlers[:]: 
        logger.removeHandler(handler)
    
    logger.setLevel(logging.INFO)
    
    file_handler = logging.FileHandler(log_filename)
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(file_handler)
    
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(console_handler)
    
    return log_filename

def train_model(model, train_loader, epochs=3, log_interval=100, save_path=None):
    log_filename = setup_logging()
    logging.info(f"Starting training with {epochs} epoch(s), log_interval={log_interval}")
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    losses = []
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch_idx, (batch_input, batch_target) in enumerate(train_loader):
            optimizer.zero_grad()
            logits, _ = model(batch_input)
            loss = criterion(logits.view(-1, vocab_size), batch_target.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
            if (batch_idx + 1) % log_interval == 0:
                avg_loss_so_far = total_loss / (batch_idx + 1)
                log_message = (f"Epoch {epoch+1}, Batch {batch_idx+1}/{len(train_loader)}, "
                              f"Batch Loss: {loss.item():.4f}, Avg Loss So Far: {avg_loss_so_far:.4f}")
                print(log_message)
                logging.info(log_message)

        avg_loss = total_loss / len(train_loader)
        losses.append(avg_loss)
        summary_message = f"Epoch {epoch+1} Summary, Avg Loss: {avg_loss:.4f}"
        print(summary_message)
        logging.info(summary_message)
    
    if save_path is not None:
        torch.save(model.state_dict(), save_path)
        logging.info(f"Model saved to {save_path}")
        print(f"Model saved to {save_path}")
    
    logging.info("Training completed")
    print(f"Training log saved to: {log_filename}")
    return losses

print("Training RNN model...")
save_path = "rnn_model_new.pth" 
rnn_losses = train_model(rnn_model, train_loader, epochs=1, log_interval=100, save_path=save_path)

2025-03-31 00:59:20,878 - INFO - Starting training with 1 epoch(s), log_interval=100


Training RNN model...


2025-03-31 00:59:25,245 - INFO - Epoch 1, Batch 100/31105, Batch Loss: 4.2583, Avg Loss So Far: 4.2274


Epoch 1, Batch 100/31105, Batch Loss: 4.2583, Avg Loss So Far: 4.2274


2025-03-31 00:59:29,610 - INFO - Epoch 1, Batch 200/31105, Batch Loss: 4.2716, Avg Loss So Far: 4.2483


...

Epoch 1, Batch 31000/31105, Batch Loss: 3.4214, Avg Loss So Far: 3.8649


2025-03-31 01:27:44,177 - INFO - Epoch 1, Batch 31100/31105, Batch Loss: 3.5912, Avg Loss So Far: 3.8641


Epoch 1, Batch 31100/31105, Batch Loss: 3.5912, Avg Loss So Far: 3.8641


2025-03-31 01:27:44,460 - INFO - Epoch 1 Summary, Avg Loss: 3.8641
2025-03-31 01:27:44,506 - INFO - Model saved to rnn_model_new
2025-03-31 01:27:44,507 - INFO - Training completed


Epoch 1 Summary, Avg Loss: 3.8641
Model saved to rnn_model_new
Training log saved to: training_log_20250331_005920.log


In [ ]:
def evaluate_perplexity(model, data_loader):
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='none')
    
    with torch.no_grad():
        for inputs, targets in data_loader:
            outputs, _ = model(inputs)
            loss = criterion(outputs.view(-1, outputs.size(-1)), targets.view(-1))
            mask = targets != 0
            batch_loss = loss[mask.view(-1)].sum().item()
            batch_tokens = mask.sum().item()
            total_loss += batch_loss
            total_tokens += batch_tokens
    
    if total_tokens == 0:
        raise ValueError("No non-padding tokens found in the dataset. Check your data or padding index.")
    
    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    print(f"Total non-padding tokens: {total_tokens}")
    print(f"Total loss: {total_loss:.4f}")
    print(f"Average loss: {avg_loss:.4f}")
    print(f"Perplexity: {perplexity:.2f}")
    return perplexity

rnn_model=RNN_LM(vocab_size,embed_size,hidden_size,num_layers,bidirectional)
rnn_model.load_state_dict(torch.load('rnn_model_new.pth'))
rnn_model.eval()
rnn_perplexity = evaluate_perplexity(rnn_model, test_loader)
print(f"RNN Test Perplexity: {rnn_perplexity:.2f}")

Total non-padding tokens: 1105920
Total loss: 3930417.2094
Average loss: 3.5540
Perplexity: 34.95
RNN Test Perplexity: 34.95


In [ ]:
def generate_text(model, prefix, vocab, max_length=50, device="cpu"):
    model.eval()
    with torch.no_grad():
        prefix_words = word_tokenize(prefix.lower())
        prefix_indices = [vocab.get(word, vocab['<UNK>']) for word in prefix_words]
        input_seq = torch.tensor(prefix_indices, dtype=torch.long).unsqueeze(0).to(device)  # Add batch dimension
        
        hidden = model.init_hidden(1, device) 
        
        # Generate sequence
        generated_indices = prefix_indices.copy()
        
        for _ in range(max_length - len(prefix_indices)):
            output, hidden = model(input_seq, hidden)
            
            last_output = output[:, -1, :]  
            next_word_idx = torch.argmax(last_output, dim=-1).item()  # Greedy search
            
            generated_indices.append(next_word_idx)
            
            input_seq = torch.tensor([next_word_idx], dtype=torch.long).unsqueeze(0).to(device)
        
        generated_words = [list(vocab.keys())[list(vocab.values()).index(idx)] for idx in generated_indices]
        return " ".join(generated_words)

prefixes = [
    "Harry looked",
    "Hermione said",
    "Ron shouted",
    "Dumbledore stood",
    "Snape glared"
]

print("\nGenerating 5 pairs of sentences (RNN):")
for prefix in prefixes:
    rnn_sentence = generate_text(rnn_model, prefix, vocab, max_length=30, device='cpu')
    
    print(f"\nPrefix: {prefix}")
    print(f"RNN: {rnn_sentence}")


Generating 5 pairs of sentences (RNN):

Prefix: Harry looked
RNN: harry looked up at the ceiling and then the door swung open and he was sure that he had not seen him .he was wearing a long overcoat and a

Prefix: Hermione said
RNN: hermione said ron nodding toward the remainder of the bus .the ministry of magic confirmed that he had been discovered to act as though he had a stitch in his

Prefix: Ron shouted
RNN: ron shouted at the dangling and carrying a large suitcase and banged her eyes and scanning it to the kilted glass of the chamber of secrets and finally the dark

Prefix: Dumbledore stood
RNN: dumbledore stood up and croaked hedwig and roger davies was almost glad to see her .aberforths not supposed to be in the forest .harry felt a thrill of foreboding .avada

Prefix: Snape glared
RNN: snape glared at him as though he had a stitch in his chest was torn and his eyes were rolling madly and down the table and shouted expelliarmusv and he
